In [ ]:
class RouteFinder():
    
    def __init__(self, name):
        self.name = name
        self.routes = set()
        
    def add_route(self, route):
        self.routes.add(route)
        
    def get_route_distance(self, towns):
        distance = 0
        # adds the distances 
        for town in towns:
            pass
        
    def get_possible_routes(self, Town1, Town2, max_stops=None, exact_stops=None, max_distance=None):
        
        # finds all possible routes between two towns, optionally with a maximum number of stops, exact number of stops, or maximum distance
        self.start = Town1
        self.end = Town2
        self.max_stops = max_stops
        self.exact_stops = exact_stops
        self.max_distance = max_distance
        
        # User should define either max_stops, exact_stops or max_distance
        if not max_stops or exact_stops or max_distance:
            print("Error: Please define either max_stops, exact_stops or max_distance")
            
        # A can never be the end point of a route thus the function returns an error if the end point is set as A
        if self.end == 'A':
            print("Error: A cannot be the end point of a route")
            
        # if user sets the exact stops to anything other than None, then the max stops is set to that value
        # this automatically cuts off the program after the exact number of stops is reached
        if exact_stops:
            self.max_stops = exact_stops
            
        possible_routes = []
        
        
    def get_shortest_route(self, Town1, Town2):
        # first finds all possible routes (without repeating), then finds the shortest one among them
        self.possible_routes = self.get_possible_routes(Town1, Town2)
        self.possible_route_distances = []
        
        # get the distances of the routes
        for route in self.possible_routes:
            self.possible_route_distances.append(self.get_route_distance(route))

        # find the shortest route out of the routes
        self.shortest_route = self.possible_routes[self.possible_route_distances.index(min(self.possible_route_distances))]
        
        print(f"the shortest route between {Town1} and {Town2} is {self.shortest_route} with a distance of {min(self.possible_route_distances)}")
        return self.shortest_route
    

In [ ]:
test_routefinder = RouteFinder("test_routefinder")
#AB5, BC4, CD8, DC8, DE6, AD5, CE2, EB3, AE7
test_routefinder.add_route("AB5")
test_routefinder.add_route("BC4")
test_routefinder.add_route("CD8")
test_routefinder.add_route("DC8")
test_routefinder.add_route("DE6")
test_routefinder.add_route("AD5")
test_routefinder.add_route("CE2")
test_routefinder.add_route("EB3")
test_routefinder.add_route("AE7")
print(test_routefinder)

In [ ]:

print(test_routefinder.routes)